### Extract Job Info

In [1]:
import pandas as pd
import chardet
import json
from tqdm import tqdm
from module.rss import RSSGetter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from openai import OpenAI

class DataGenerator:
    def __init__(self):
        self.openai_api_key = "EMPTY"
        self.openai_api_base = "http://192.168.1.20:1318/v1"
        self.client = OpenAI(
            api_key=self.openai_api_key,
            base_url=self.openai_api_base,
        )
        self.USE_MODEL = "pixtral"
        self.MODEL_NAME_DICT = {
            "qwen": "Qwen/Qwen2-VL-72B-Instruct",
            "pixtral": "mistralai/Pixtral-Large-Instruct-2411"
        }
        self.MODEL_NAME = self.MODEL_NAME_DICT[self.USE_MODEL]
        self.SYSTEM_PROMPT_DICT = {
            "qwen": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
            "pixtral": "You are a helpful assistant."
        }
        self.SYSTEM_PROMPT = self.SYSTEM_PROMPT_DICT[self.USE_MODEL]

    def LLM_Call(self, 
                 prompt: str) -> str:
        chat_response = self.client.chat.completions.create(
            model=self.MODEL_NAME,
            messages=[
                {"role": "system", "content": self.SYSTEM_PROMPT},
                {"role": "user", "content": f"{prompt}"},
            ],
            temperature=0.001,
            top_p=0.001,
            max_tokens=4096,
            extra_body={
                "repetition_penalty": 1.03,
            },
        )
        return chat_response.choices[0].message.content

In [3]:
def get_question(job_name: str,
                 question_type: str) -> str:
    prompt = f"""당신은 데이터 생성 전문가입니다.
    
    [직업 이름]과 [질문 종류]를 참고하여, 질문 또는 요청문을 1개 생성해주세요.
    무조건 질문 또는 요청문만 생성합니다.
    질문 또는 요청문은 무조건 한국어로 생성합니다. 
    
    예를들어,
    1. **가 하는일은 무엇인가요?
    2. **의 직무에 대해서 알려주세요.
    3. **의 연봉은 얼마쯤인가요?
    4. **의 전망은 어떤가요?
    5. **의 임금 상승률은 어떤가요?
    6. **가 하는일에 대해 알려주세요.
    7. **의 연봉에 대해 알려주실래요?
    8. **의 전망에 대해 알려주세요.
    
    [직업 이름]
    {job_name}
    
    [질문 종류]
    {question_type}
    """
    return prompt

def get_answer(job_name: str,
               question: str) -> str:
    prompt = f"""당신은 데이터 생성 전문가입니다.
    
    [직업 이름]과 [질문]을 참고하여, 답변을 생성해주세요.
    답변은 무조건 한국어로 생성합니다.
    
    [직업 이름]
    {job_name}
    
    [질문]
    {question}
    """
    return prompt

In [4]:
job_data_generator = DataGenerator()
rss = RSSGetter()

🖥ChromeDriver가 존재합니다.


데이터 종류
- 1. 직무, 하는일 (Job Description, Job Duties)
- 2. 임금, 연봉 (Salary, Annual Salary)
- 3. 전망 (Job Outlook, Job Growth)

In [5]:
encoding = chardet.detect(open('job_data.csv', 'rb').read())
df = pd.read_csv('job_data.csv',
                 encoding=encoding['encoding'])
job_list = list(df['KNOW직업명'].values)

In [ ]:
JSON_DATA = []
for job in tqdm(job_list):
    print(f"직업이름 : {job}")
    try:
        for question_type in ["직무, 하는일", "임금, 연봉", "전망"]:
            question_prompt = get_question(job, question_type).strip()
            question = job_data_generator.LLM_Call(question_prompt).strip()
            answer_prompt = get_answer(job, question).strip()
            answer = job_data_generator.LLM_Call(answer_prompt).strip()
            JSON_DATA.append({
                "job_name": job,
                "question_type": question_type,
                "question_prompt": question_prompt,
                "question": question,
                "answer_prompt": answer_prompt,
                "answer": answer
            })
    except:
        print(f"Error: {job}")
        continue
    with open("job_data.json", "w", encoding="utf-8") as f:
        json.dump(JSON_DATA, f, ensure_ascii=False, indent=4)